In [411]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split 
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.tree import DecisionTreeClassifier
from binarytree import tree,Node
import copy

In [412]:
df=pd.read_csv("train.csv")

In [413]:
X=df.drop(['left'],axis=1)
Y=df['left']
X = pd.concat([X,pd.get_dummies(X['sales'], prefix='sales')],axis=1)
X = pd.concat([X,pd.get_dummies(X['salary'], prefix='salary')],axis=1)
X.drop(['sales','salary'],axis=1, inplace=True)

In [414]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size = 0.2)
X_Original_train=copy.deepcopy(X_train)
X_train = pd.concat([X_train,Y_train],axis=1)
X_Original_test=copy.deepcopy(X_test)
eps = np.finfo(float).eps
# X_train

In [415]:
def find_overall_entropy(X_train):
    output=X_train.keys()[-1]
    target_value_list=X_train[output].unique()
    entropy=0
    for value in target_value_list:
        frac = float(X_train[output].value_counts()[value])/(len(X_train[output])+eps)
        entropy+=-frac*np.log2(frac+eps)
    return entropy

In [416]:
def find_attr_entropy(X_train,attr):
    if X_train.empty:
        return
    output=X_train.keys()[-1]
    target_value_list=X_train[output].unique()
    current_value_list=X_train[attr].unique()
    final_entropy=0
    den=1
    for cur_value in current_value_list:
        entropy=0
        for value in target_value_list:
            num = len(X_train[attr][X_train[attr] == cur_value][X_train[output] == value])
            den = len(X_train[attr][X_train[attr] == cur_value])
            frac=num/(den+eps)
            entropy+=-frac*np.log2(frac+eps)
        frac2=float(den)/len(X_train)
        final_entropy+=-frac2*entropy
    return final_entropy

In [417]:
def get_subdataframeNum1(X_train,node,value):
    if X_train.empty:
        return
    return X_train[X_train[node] < value].reset_index(drop=True)

In [418]:
def get_subdataframeNum2(X_train,node,value):
    if X_train.empty:
        return
    return X_train[X_train[node] >= value].reset_index(drop=True)

In [419]:
storemidpoint={}
storemidpoint.clear()
numeric_features = ['number_project','last_evaluation','satisfaction_level','average_montly_hours','time_spend_company']
for feature in numeric_features:
    IGMax=0
    midmax=0
    prev=None
    X_train.sort_values(feature,inplace=True)
    for index,row in X_train.iterrows():
        if prev is not None:
            if prev['left']!=row['left']:
                mid=(float(prev[feature])+float(row[feature]))/2
    #             print mid
                subtable1=get_subdataframeNum1(X_train,feature,mid)
                subtable2=get_subdataframeNum2(X_train,feature,mid)
                firstentropy=0
                secondentropy=0
                if subtable1.empty:
                    firstentropy=0
                else:
                    frac1=float(len(subtable1))/len(X_train)
                    firstentropy = frac1*find_overall_entropy(subtable1)
                if subtable2.empty:
                    secondentropy=0
                else:
                    frac2=float(len(subtable2))/len(X_train)
                    secondentropy = frac2*find_overall_entropy(subtable2)
                
                entropy1=find_overall_entropy(X_train)
                entropy2=firstentropy+secondentropy
                ig=abs(entropy1)-abs(entropy2)
    #             print ig
                if ig > IGMax:
                    IGMax=ig
                    midmax=mid

        prev=row
        storemidpoint[feature]=midmax
    for index,row in X_train.iterrows():
        if float(row[feature])>=midmax:
            X_train.at[index,feature]=1
        else:
            X_train.at[index,feature]=0
# print IGMax,midmax

In [420]:
def find_best_node(X_train):
    if X_train.empty:
        return
    information_gain=[]
    for attr in X_train.keys()[:-1]:
        information_gain.append(find_overall_entropy(X_train)+find_attr_entropy(X_train,attr))
    return X_train.keys()[:-1][np.argmax(information_gain)]

In [421]:
def get_subdataframe(X_train,node,value):
    if X_train.empty:
        return
    return X_train[X_train[node] == value].reset_index(drop=True)

In [422]:
class nodestruct:
    def __init__(self, value, left=None, right=None):
        self.value=value
        self.left=left
        self.right=right

In [423]:
def build_tree(X_train):
    if X_train.empty:
        return
    if len(X_train.columns)==1:
        return 
    output=X_train.keys()[-1]
    node = find_best_node(X_train)
    
    root=nodestruct(node)
#     print root.value
    
    attValue=X_train[node].unique()
    for val in attValue:

        subdataframe = get_subdataframe(X_train,node,val)
        outputValue,counts = np.unique(subdataframe[output],return_counts=True)                        
        subdataframe=subdataframe.drop([node],axis=1)

        if len(counts)<=1:
            root.value=outputValue[0]
            return root
        else:
            if val == 0:
                root.left = build_tree(subdataframe)
            else:
                root.right = build_tree(subdataframe)
    
    return root

In [424]:
rootNode=build_tree(X_train)
print rootNode.value

satisfaction_level


In [425]:
def testing(X_test,root):
    if X_test.empty:
        return
    if root==None:
        return
    if root.value == 0 or root.value == 1:
        y_pred.append(root.value)
#         print root.value
        return
    features = X_test.keys()
#     print features
    for j in features:
        if j == root.value:
#             X_test=X_test.drop([j],axis=1)
            if X_test[j] == '0':
                testing(X_test,root.left)
            else:
                testing(X_test,root.right)

In [426]:
for feature in numeric_features:
    for index,row in X_test.iterrows():
        if float(row[feature]) >= storemidpoint[feature]:
            X_test.at[index,feature]=1
        else:
            X_test.at[index,feature]=0


In [427]:
y_pred=[]
print rootNode.value
for index,row in X_test.iterrows():
    testing(row,rootNode)
print len(y_pred)

print confusion_matrix(Y_test,y_pred)
print classification_report(Y_test,y_pred)
print accuracy_score(Y_test, y_pred)

satisfaction_level
2248
[[1719    0]
 [ 529    0]]
              precision    recall  f1-score   support

           0       0.76      1.00      0.87      1719
           1       0.00      0.00      0.00       529

   micro avg       0.76      0.76      0.76      2248
   macro avg       0.38      0.50      0.43      2248
weighted avg       0.58      0.76      0.66      2248

0.7646797153024911


In [428]:
modelTrain = DecisionTreeClassifier()
modelTrain.fit(X_Original_train,Y_train)
Y_predict=modelTrain.predict(X_Original_test)
print confusion_matrix(Y_test,Y_predict)
print classification_report(Y_test,Y_predict)
print accuracy_score(Y_test, Y_predict)

[[1684   35]
 [  22  507]]
              precision    recall  f1-score   support

           0       0.99      0.98      0.98      1719
           1       0.94      0.96      0.95       529

   micro avg       0.97      0.97      0.97      2248
   macro avg       0.96      0.97      0.97      2248
weighted avg       0.97      0.97      0.97      2248

0.974644128113879
